In [ ]:
!pip3.10 install nltk

In [ ]:
!pip3.10 install xlsxwriter

In [6]:
import random
random.seed(42)

import numpy as np
np.random.seed(42)

import torch
torch.manual_seed(42)


import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
from os import listdir
from os.path import isfile, join

mypath = "review_analysis"
jsons = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]
len(jsons)

113

In [9]:
ru_names = [file.split('.')[0].split('/')[1] for file in jsons]
len(set(ru_names))

105

In [10]:
import json
from sklearn.preprocessing import LabelEncoder
import numpy as np

from collections import defaultdict

data = defaultdict(list)


#data = []
labels = []
ru_plains = []
en_plains = []
names_len_labs = {}
cnt = 0


for j in jsons:
    try:
        with open('review_analysis_translated/' + j.split('/')[1][:-5] + '.ann') as of:
            lines = of.read().splitlines()
        for line in lines:
            if line.split('\t')[0].split()[1] == 'relevance':
                data[j].append([line.split('\t')[1], 'Актуальность'])
                labels.append('Актуальность')
            elif line.split('\t')[0].split()[1] == 'characteristics':
                data[j].append([line.split('\t')[1], 'Характеристики метода'])
                labels.append('Характеристики метода')
            else:
                data[j].append([line.split('\t')[1], 'Новизна'])
                labels.append('Новизна')
            
    except FileNotFoundError:
        print(j)
        continue
    with open(j,"r") as json_data:
        d = json.load(json_data)          
        plain = d['plain_text']
 
        cnt_lab = 0

        f = d["frames"]
        for frame_dict in f: #'slots', 'type', 'name', 'id']
            for cues_dict in (frame_dict['slots']):
                for c in cues_dict['cues']:
                    plain = plain.replace(c['text'],"")
                    if c['text'].find("$") ==-1 and c['text'].find('\ue016')==-1:
                        if [c['text'], frame_dict['type']] in data[j]:
                            continue
                        else:
                            data[j].append([c['text'],frame_dict['type']])
                            cnt_lab +=1
                            labels.append(frame_dict['type'])
        if cnt < 55:
            ru_plains.append(plain) 
            cnt+=1
        else:
            with open('review_analysis_translated/' + j.split('/')[1][:-5] + '.txt') as of:
                text = of.read().strip()
                en_plains.append(text)

review_analysis/proj_Рецензии статей_subj_ИИПР_doc_21-10-04_2 на ст. Звонарева, авт..json
review_analysis/proj_Рецензии статей_subj_ИТВС_doc_Рецензия Каратач.json
review_analysis/proj_Рецензии статей_subj_ИТВС_doc_Рецензия Орлова_3-2023.json


In [11]:
data[jsons[90]]

[['The most important directions including those implemented are not fully reflected in particular AI is now widely used and provides qualitatively new results in such areas as automatic analysis of X rays MRI ECG blood analysis epidemiological studies disease diagnosis including infectious diseases and others for example the IDDAP system',
  'Новизна'],
 ['Accordingly the originality of the results raises some doubt as does the comparison with already existing articles',
  'Новизна'],
 ['Не полностью отражены важнейшие направления в т . ч . реализованные , в частности ИИ сейчас широко применяется и дает качественно новые результаты в таких областях , как : автоматический анализ рентгеновских снимков , МРТ , ЭКГ , анализ крови , эпидемиологические исследования , диагностика заболеваний , в т . ч . инфекционных и пр ., например система IDDAP',
  'Новизна'],
 ['Соответственно , оригинальность результатов вызывает некоторое сомнение , как и сравнение с уже имеющимися статьями',
  'Новизна

In [12]:
data = list(data.values())

In [13]:
from sklearn.model_selection import train_test_split

dtrain, dtest = train_test_split(data, test_size=0.25, train_size=0.75, random_state=2025, shuffle = True)    

In [14]:
full_data_train = []
for data_list in dtrain:
    for elem in data_list:
        full_data_train.append(elem)

In [15]:
from collections import Counter
labels = labels + ['Иное']*150
Counter(labels)

Counter({'Характеристики метода': 220,
         'Новизна': 190,
         'Иное': 150,
         'Актуальность': 118})

In [16]:
le = LabelEncoder()
le.fit(labels)

for do in full_data_train:
    do[1] = le.transform(np.asarray([do[1]]))[0] 

In [17]:
full_data_test = []
for data_list in dtest:
    for elem in data_list:
        full_data_test.append(elem)
        
for do in full_data_test:
    do[1] = le.transform(np.asarray([do[1]]))[0] 

In [18]:
for i,p in enumerate(ru_plains):
    com_offset = p.find("Комментарии")
    deny_offset = p.find("укажите причины отклонения")
    if com_offset > deny_offset:
        ru_plains[i] = p[com_offset + len("Комментарии"):]
    else:
        ru_plains[i] = p[deny_offset + len("укажите причины отклонения"):]

In [19]:
for i,p in enumerate(en_plains):
    com_offset = p.find('Comments')
    deny_offset = p.find("specify the reasons for rejection")
    if com_offset > deny_offset:
        en_plains[i] = p[com_offset + len("Comments"):]
    else:
        en_plains[i] = p[deny_offset + len("specify the reasons for rejection"):]


In [20]:
from nltk.tokenize import sent_tokenize

ru_plains = [".".join(p.split(".")) for p in ru_plains]        
ru_plains = [" ".join(p.split()) for p in ru_plains]    
  
inoe_texts = []    
    
for p in ru_plains:
    texts = sent_tokenize(p)
    for t in texts:
        inoe_texts.append([t,"Иное"])
        
en_plains = [".".join(p.split(".")) for p in en_plains]        
en_plains = [" ".join(p.split()) for p in en_plains]    
  
for p in en_plains:
    texts = sent_tokenize(p)
    for t in texts:
        inoe_texts.append([t,"Иное"])

In [21]:
random_ints = np.random.choice(np.arange(len(inoe_texts)), size=200, replace=False).tolist()
inoe_texts = np.array(inoe_texts)[random_ints].tolist()

for do in inoe_texts:
    do[1] = le.transform(np.asarray([do[1]]))[0] 


In [22]:
inoe_train, inoe_test = train_test_split(inoe_texts, test_size=0.25, train_size=0.75, random_state=2025, shuffle = True) 

In [23]:
full_data_train = full_data_train + inoe_train
full_data_test = full_data_test + inoe_test

In [24]:
import pandas
from sklearn.model_selection import train_test_split

#dtrain, dtest = train_test_split(clean_data, test_size=0.2, train_size=0.8, random_state=2025, stratify=labs, shuffle = True)    
    
pandas.DataFrame(full_data_train,columns=['text','label']).to_csv("train_seg_clean.csv", index=False)    
pandas.DataFrame(full_data_test,columns=['text','label']).to_csv("test_seg_clean.csv", index=False)  

In [25]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

dataset = load_dataset('csv', data_files={"train": "train_seg_clean.csv", "test": "test_seg_clean.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
#tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = dataset.map(preprocess_function, batched=True).remove_columns("text").with_format("torch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors = 'pt')

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

In [27]:
import torch
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

accuracy = evaluate.load("accuracy")

model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=len(le.classes_))

#model = AutoModelForSequenceClassification.from_pretrained(
#    "ai-forever/ruBert-base", num_labels=len(le.classes_)).to(device)

comet_ml is installed but `COMET_API_KEY` is not set.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
#    return accuracy.compute(predictions=predictions, references=labels)
    return f1.compute(predictions=predictions, references=labels, average='macro')

training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    save_total_limit = 2,
    no_cuda=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [31]:
prec = evaluate.load("precision")
rec = evaluate.load("recall")
f1 = evaluate.load("f1")

In [32]:
#seeds = [42, 2025, 768, 893, 324]

seeds=[42]

f1s, precs, recs = [], [], []
for seed in seeds:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    print('seed equals ', str(seed))
    
    model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base", num_labels=len(le.classes_))
    
    
    training_args = TrainingArguments(
        output_dir="rev-classif-twolangs",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=8,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=True,
        save_total_limit = 2,
        no_cuda=True
    )
        
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics)  
    
    print('Training')
    trainer.train()
    
    
    print('Compute metrics')
    predictions = trainer.predict(tokenized_ds["test"])
    preds = np.argmax(predictions.predictions, axis=-1)
    
    f1s.append(f1.compute(predictions=preds, references=predictions.label_ids,average='macro')['f1'])
    recs.append(rec.compute(predictions=preds, references=predictions.label_ids,average='macro')['recall'])
    precs.append(prec.compute(predictions=preds, references=predictions.label_ids,average='macro')['precision'])
    

seed equals  42


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training


Epoch,Training Loss,Validation Loss,F1
1,No log,1.246490,0.283595
2,No log,0.967337,0.474769
3,No log,0.901721,0.587567
4,No log,0.703818,0.775779
5,No log,0.574542,0.801270
6,No log,0.567421,0.819953
7,No log,0.569793,0.798791
8,No log,0.561004,0.822881


Compute metrics


xlm-roberta-large

f1s: [0.7280025284450063, 0.2782418184307819, 0.786073787074856, 0.7149246102734474, 0.7446538543383738]
recs: [0.7798390151515151, 0.335, 0.8081060606060606, 0.7498674242424244, 0.7712310606060606]
precs: [0.7157007581489664, 0.35855263157894735, 0.7758261494252874, 0.7136005664567924, 0.7420385986562457]

xlm-roberta-base

f1s: [0.8228809442281555, 0.6834353358723498, 0.7024806991914458, 0.7600666990055784, 0.6767720343243773]
recs: [0.848967803030303, 0.7269886363636364, 0.7416003787878788, 0.7693939393939393, 0.7194128787878789]
precs: [0.8095238095238095, 0.6785097335097334, 0.6860720411663808, 0.7649261470773099, 0.6656557452593258]

google-bert/bert-base-multilingual-uncased

f1s:   [0.6955929959304537, 0.7799337200065084, 0.4832565051471278, 0.7856242450200102, 0.7879278454241344]
recs:  [0.7181628787878788, 0.8036268939393939, 0.5067803030303031, 0.815965909090909, 0.7988920454545454]
precs: [0.6830876520359598, 0.771662210338681, 0.4679861111111111, 0.7714697376995765, 0.7791094619666048]




In [69]:
print(recs)

[0.7181628787878788, 0.8036268939393939, 0.5067803030303031, 0.815965909090909, 0.7988920454545454]


In [31]:
trainer.save_model("my_model")

In [33]:
predictions = trainer.predict(tokenized_ds["test"])
preds = np.argmax(predictions.predictions, axis=-1)


In [34]:
prec.compute(predictions=preds, references=predictions.label_ids,average='macro')['precision']

0.8095238095238095

In [35]:
rec.compute(predictions=preds, references=predictions.label_ids,average='macro')['recall']

0.848967803030303

In [36]:
f1.compute(predictions=preds, references=predictions.label_ids,average='macro')['f1']

0.8228809442281555

In [36]:
preds = np.argmax(predictions.predictions, axis=-1)

In [37]:
from sklearn.metrics import classification_report

#predictions = trainer.predict(tokenized_ds["test"])

print(classification_report(
    y_true=tokenized_ds["test"]['label'],
    y_pred=preds,
    target_names=['Актуальность', 'Иное', 'Новизна', 'Характеристики метода']
))

                       precision    recall  f1-score   support

         Актуальность       0.70      0.95      0.81        22
                 Иное       0.93      0.84      0.88        50
              Новизна       0.77      0.84      0.81        32
Характеристики метода       0.83      0.76      0.79        66

             accuracy                           0.82       170
            macro avg       0.81      0.85      0.82       170
         weighted avg       0.83      0.82      0.82       170



In [44]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)

#predictions = trainer.predict(tokenized_ds["test"])

print(classification_report(
    y_true=tokenized_ds["train"]['label'],
    y_pred=preds,
    target_names=['Актуальность', 'Иное', 'Новизна', 'Характеристики метода']
))

                       precision    recall  f1-score   support

         Актуальность       0.96      0.97      0.96        96
                 Иное       1.00      1.00      1.00       150
              Новизна       0.87      0.94      0.90       158
Характеристики метода       0.92      0.83      0.87       154

             accuracy                           0.93       558
            macro avg       0.94      0.94      0.94       558
         weighted avg       0.93      0.93      0.93       558



In [ ]:
le.classes_

In [ ]:
prec.compute(predictions=preds, references=predictions.label_ids,average='macro')

# Generate output for humans

In [45]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm, trange


tdata = []
for j in tqdm(jsons):
    with open(j,"r") as json_data:
        d = json.load(json_data)    
        f = d["frames"]
        for frame_dict in f: #'slots', 'type', 'name', 'id']
            for cues_dict in (frame_dict['slots']):
                for c in cues_dict['cues']:
                    if c['text'].find("$") ==-1 and c['text'].find('\ue016')==-1: 
                        text = c['text']
                        tokens = tokenizer(text, truncation=True)
                        predictions = trainer.predict([tokens])
                        tdata.append([j,text,le.inverse_transform(np.argmax(predictions.predictions, axis=-1)),frame_dict['type']]) 
                        
df = pd.DataFrame(tdata,columns=["Файл","Текст","Машина","Разметчик"])
df.to_excel("review_segmentation.xlsx", engine='xlsxwriter')                        

  0%|          | 0/113 [00:00<?, ?it/s]

  2%|▏         | 2/113 [00:01<01:28,  1.26it/s]

  3%|▎         | 3/113 [00:01<01:01,  1.80it/s]

  4%|▎         | 4/113 [00:02<00:50,  2.17it/s]

  4%|▍         | 5/113 [00:02<00:51,  2.11it/s]

...

 95%|█████████▍| 107/113 [02:39<00:07,  1.19s/it]

 96%|█████████▌| 108/113 [02:40<00:05,  1.18s/it]

 96%|█████████▋| 109/113 [02:40<00:03,  1.10it/s]

 97%|█████████▋| 110/113 [02:43<00:04,  1.47s/it]

100%|██████████| 113/113 [02:43<00:00,  1.45s/it]
